In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.utils.path_converter import path_converter

df = pd.read_csv(path_converter("/data/raw/dates_100k.csv"))

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

train_df.to_csv(path_converter("/data/raw/train_dates.csv"), index=False)
val_df.to_csv(path_converter("/data/raw/val_dates.csv"), index=False)
test_df.to_csv(path_converter("/data/raw/test_dates.csv"), index=False)

In [6]:
from datasets import load_dataset
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")


def preprocess(example):
    model_inputs = tokenizer(
        example["input_text"], max_length=64, padding="max_length", truncation=True
    )
    labels = tokenizer(
        example["target_text"], max_length=16, padding="max_length", truncation=True
    )["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs


dataset = load_dataset(
    "csv",
    data_files={
        "train": path_converter("/data/raw/train_dates.csv"),
        "val": path_converter("/data/raw/val_dates.csv"),
        "test": path_converter("/data/raw/test_dates.csv"),
    },
)

dataset = dataset.map(preprocess)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Generating train split: 81000 examples [00:00, 1161257.39 examples/s]
Generating val split: 9000 examples [00:00, 1423138.02 examples/s]
Generating test split: 10000 examples [00:00, 1344069.73 examples/s]
Map: 100%|██████████| 10000/10000 [00:01<00:00, 7918.36 examples/s]


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import TrainingArguments, Trainer

model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="date-normalizer",
    num_train_epochs=4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=3e-4,
    logging_steps=200,
    use_cpu=True,
    no_cuda=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
)

trainer.train(resume_from_checkpoint="date-normalizer/checkpoint-3000")


In [ ]:
def normalize_date(text: str):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=20, num_beams=5)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


print(normalize_date("2010-12-01 08:26:00"))


2010-12-01


In [10]:
import numpy as np


def evaluate_model(model, tokenizer, dataset):
    correct = 0
    total = len(dataset)
    i = 0

    for sample in dataset:
        i += 1
        inp = tokenizer(sample["input_text"], return_tensors="pt")
        output = model.generate(**inp, max_length=20)
        prediction = tokenizer.decode(output[0], skip_special_tokens=True)

        if prediction == sample["target_text"]:
            correct += 1
        print(correct / i)

    print("Accuracy:", correct / total)

In [11]:
evaluate_model(model, tokenizer, dataset["test"])

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9
0.9090909090909091
0.9166666666666666
0.9230769230769231
0.9285714285714286
0.9333333333333333
0.9375
0.9411764705882353
0.8888888888888888
0.8421052631578947
0.85
0.8095238095238095
0.7727272727272727
0.782608695652174
0.75
0.76
0.7692307692307693
0.7777777777777778
0.7857142857142857
0.7931034482758621
0.7666666666666667
0.7741935483870968
0.78125
0.7575757575757576
0.7647058823529411
0.7714285714285715
0.7777777777777778
0.7837837837837838
0.7894736842105263
0.7948717948717948
0.775
0.7804878048780488
0.7857142857142857
0.7906976744186046
0.7954545454545454
0.8
0.8043478260869565
0.8085106382978723
0.7916666666666666
0.7959183673469388
0.8
0.803921568627451
0.7884615384615384
0.7924528301886793
0.7962962962962963
0.8
0.8035714285714286
0.8070175438596491
0.8103448275862069
0.8135593220338984
0.8166666666666667
0.819672131147541
0.8064516129032258
0.8095238095238095
0.8125
0.8
0.803030303030303
0.8059701492537313
0.8088235294117647
0.8115942028

KeyboardInterrupt: 

In [ ]:
model.save_pretrained("model-small", safe_serialization=True)